<a href="https://colab.research.google.com/github/vloneonme/trew/blob/main/dm_lr3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from collections import Counter
import math

with open('4_6_kapitanskaya_do4ka.txt', 'r', encoding='utf-8') as file1:  # прочитали файлы и записали их в str
    dochka = file1.read()
with open('4_6_travel.txt', 'r', encoding='utf-8') as file2:
    puteshestvie = file2.read()

n_dochka = len(dochka)
n_puteshestvie = len(puteshestvie)

freq_dochka = dict(Counter(dochka).most_common())
freq_puteshestvie = dict(Counter(puteshestvie).most_common())


for char, freq in freq_dochka.items():
    freq_dochka[char] = freq/n_dochka

for char, freq in freq_puteshestvie.items():
    freq_puteshestvie[char] = freq/n_puteshestvie
    # print(f'{repr(char)}: {round(freq/n_puteshestvie, 5)}')

entropy_dochka = 0
entropy_puteshestvie = 0

for p in freq_dochka.values():
    entropy_dochka -= p*math.log2(p)

for p in freq_puteshestvie.values():
    entropy_puteshestvie -= p*math.log2(p)



def uniform_code(distribution_freq):
    n = len(distribution_freq)  # колво символов
    encoding = {}
    bits = math.ceil(math.log2(n))  # длина слова ceil округляет в большую сторону
    for i, char in enumerate(distribution_freq.keys()):
        encoding[char] = format(i, f'0{bits}b')  # переводим число i в бинарную запись,
        # 0 - заполнить нулями, {bits} длина, b - бинарная запись
    return encoding


def shannon_fano_code(distribution_freq, current_str = '', encoding = {}):
    n = len(distribution_freq)
    if n == 1:  # остался последний элемент, значит код готов
        encoding[list(distribution_freq)[0]] = current_str
        return
    min_diff = 1000000
    split_index = 0
    for i in range(n):  # делим на две группы так, чтобы они были равновероятны
        curr_diff = abs(sum(list(distribution_freq.values())[i:]) - sum(list(distribution_freq.values())[:i]))
        if curr_diff < min_diff:
            min_diff = curr_diff
            split_index = i
    group1 = dict(list(distribution_freq.items())[split_index:])
    group2 = dict(list(distribution_freq.items())[:split_index])

    # рекурсируем продолжая алгоритм пока не дойдем до одного эл-та в группе
    shannon_fano_code(group1, current_str+'0', encoding)
    shannon_fano_code(group2, current_str+'1', encoding)

# result = {}
# shannon_fano_code(freq_dochka, '', result)

def huffman_code(distribution_freq): # каждый символ зададим как листовые вершинки дерева хаффмана
    # Список списков вершин и их частот
    nodes = [[freq, char] for char, freq in distribution_freq.items()]
    encoding = {char: "" for char in distribution_freq.keys()}

    while len(nodes) > 1:
        nodes.sort(key=lambda x: x[0])
        left = nodes.pop(0)  # самые маловероятные две вершины в будущем дереве хаффмана
        right = nodes.pop(0)

        for char in left[1]:  # всем символам что находятся в левой вершине добавляем '0'
            encoding[char] = '0' + encoding[char]

        for char in right[1]:
            encoding[char] = '1' + encoding[char]

        new_freq = left[0] + right[0]  # соединяем те вершины в одну старшую => частота складывается
        new_chars = left[1] + right[1]
        nodes.append([new_freq, new_chars])

    return encoding



#######################################################################################
print("=" * 50)
print("ПУНКТ 2: РАСПРЕДЕЛЕНИЕ ОТНОСИТЕЛЬНЫХ ЧАСТОТ")
print("=" * 50)

print("\n Текст 'Путешествие':")
for char, freq in sorted(freq_puteshestvie.items(), key=lambda x: x[1], reverse=True)[:15]:  # топ-15
    print(f"  '{char}': {freq:.4f}")

print("\n Текст 'Дочка':")
for char, freq in sorted(freq_dochka.items(), key=lambda x: x[1], reverse=True)[:15]:  # топ-15
    print(f"  '{char}': {freq:.4f}")

print("\n" + "=" * 50)
print("ПУНКТ 3: ВЫБОРОЧНАЯ ЭНТРОПИЯ")
print("=" * 50)

print(f"Энтропия 'Путешествие': {entropy_puteshestvie:.4f} бит")
print(f"Энтропия 'Дочка': {entropy_dochka:.4f} бит")


# Вычисляем все средние длины
def average_code_length(frequencies, encoding):
    total_length = 0.0
    for char, freq in frequencies.items():
        code_length = len(encoding[char])
        total_length += freq * code_length
    return total_length


shannon1 = {}
shannon2 = {}
shannon_fano_code(freq_dochka, '', shannon1)
shannon_fano_code(freq_puteshestvie, '', shannon2)

results = {}
# Текст "Путешествие"
results['puteshestvie_uniform'] = average_code_length(freq_puteshestvie, uniform_code(freq_puteshestvie))
results['puteshestvie_shannon'] = average_code_length(freq_puteshestvie, shannon2)
results['puteshestvie_huffman'] = average_code_length(freq_puteshestvie, huffman_code(freq_puteshestvie))

# Текст "Дочка"
results['dochka_uniform'] = average_code_length(freq_dochka, uniform_code(freq_dochka))
results['dochka_shannon'] = average_code_length(freq_dochka, shannon1)
results['dochka_huffman'] = average_code_length(freq_dochka, huffman_code(freq_dochka))

# Вывод результатов
print("\n" + "=" * 50)
print("ПУНКТ 5: СРЕДНИЕ ДЛИНЫ КОДОВЫХ СЛОВ")
print("=" * 50)
print("\n Текст 'Путешествие':")
print(f"  Равномерный: {results['puteshestvie_uniform']:.4f}")
print(f"  Шеннон-Фано: {results['puteshestvie_shannon']:.4f}")
print(f"  Хаффман:     {results['puteshestvie_huffman']:.4f}")

print("\n Текст 'Дочка':")
print(f"  Равномерный: {results['dochka_uniform']:.4f}")
print(f"  Шеннон-Фано: {results['dochka_shannon']:.4f}")
print(f"  Хаффман:     {results['dochka_huffman']:.4f}")








########################################################################

def encode_text(text, encoding_dict):
    encoded = []
    for char in text:
        encoded.append(encoding_dict[char])
    return ''.join(encoded)

def encode_text_with_missing(text, encoding_dict, text_name, code_name):
    encoded = []
    missing_chars = set()

    for char in text:
        if char in encoding_dict:
            encoded.append(encoding_dict[char])
        else:
            missing_chars.add(char)
            # Можно использовать специальный код или пропустить
            # encoded.append('000000')  # например, специальный код для неизвестных символов

    if missing_chars:
        print(f"Предупреждение: в тексте '{text_name}' при кодировании '{code_name}' не найдены символы: {missing_chars}")
    return ''.join(encoded)

print('\n'+"=" * 70)
print("ПУНКТ 6: КОДИРОВАНИЕ ТЕКСТОВ")
print("=" * 70)

# Получаем все кодировки
uniform_put = uniform_code(freq_puteshestvie)
uniform_doch = uniform_code(freq_dochka)
huffman_put = huffman_code(freq_puteshestvie)
huffman_doch = huffman_code(freq_dochka)

# Кодируем тексты
encoded_put_uniform = encode_text(puteshestvie, uniform_put)
encoded_put_shannon = encode_text(puteshestvie, shannon2)
encoded_put_huffman = encode_text(puteshestvie, huffman_put)

encoded_doch_uniform = encode_text(dochka, uniform_doch)
encoded_doch_shannon = encode_text(dochka, shannon1)
encoded_doch_huffman = encode_text(dochka, huffman_doch)

# Выводим результаты
print("\n ТЕКСТ 'ПУТЕШЕСТВИЕ':")
print(f"Исходная длина: {n_puteshestvie} символов")
print(f"Равномерный код: {len(encoded_put_uniform)} бит (первые 200 бит: {encoded_put_uniform[:200]}...)")
print(f"Шеннон-Фано:     {len(encoded_put_shannon)} бит (первые 200 бит: {encoded_put_shannon[:200]}...)")
print(f"Хаффман:         {len(encoded_put_huffman)} бит (первые 200 бит: {encoded_put_huffman[:200]}...)")

print("\n ТЕКСТ 'ДОЧКА':")
print(f"Исходная длина: {n_dochka} символов")
print(f"Равномерный код: {len(encoded_doch_uniform)} бит (первые 200 бит: {encoded_doch_uniform[:200]}...)")
print(f"Шеннон-Фано:     {len(encoded_doch_shannon)} бит (первые 200 бит: {encoded_doch_shannon[:200]}...)")
print(f"Хаффман:         {len(encoded_doch_huffman)} бит (первые 200 бит: {encoded_doch_huffman[:200]}...)")

# Сравнение эффективности на практике
print("\n ПРАКТИЧЕСКАЯ ЭФФЕКТИВНОСТЬ (бит/символ):")
print("Текст 'Путешествие':")
print(f"  Равномерный: {len(encoded_put_uniform)/n_puteshestvie:.4f}")
print(f"  Шеннон-Фано: {len(encoded_put_shannon)/n_puteshestvie:.4f}")
print(f"  Хаффман:     {len(encoded_put_huffman)/n_puteshestvie:.4f}")

print("Текст 'Дочка':")
print(f"  Равномерный: {len(encoded_doch_uniform)/n_dochka:.4f}")
print(f"  Шеннон-Фано: {len(encoded_doch_shannon)/n_dochka:.4f}")
print(f"  Хаффман:     {len(encoded_doch_huffman)/n_dochka:.4f}")



##################################################################
print("=" * 70)
print("ПУНКТ 9: КОДИРОВАНИЕ ТЕКСТОВ 'ЧУЖИМИ' КОДАМИ")
print("=" * 70)

# Кодируем с обработкой ошибок
encoded_put_with_doch_shannon = encode_text_with_missing(puteshestvie, shannon1, "Путешествие", "Шеннон-Фано от Дочки")
encoded_put_with_doch_huffman = encode_text_with_missing(puteshestvie, huffman_doch, "Путешествие", "Хаффман от Дочки")

encoded_doch_with_put_shannon = encode_text_with_missing(dochka, shannon2, "Дочка", "Шеннон-Фано от Путешествия")
encoded_doch_with_put_huffman = encode_text_with_missing(dochka, huffman_put, "Дочка", "Хаффман от Путешествия")

print("\nРАЗМЕРЫ ЗАКОДИРОВАННЫХ ТЕКСТОВ 'ЧУЖИМИ' КОДАМИ:")
print(f"'Путешествие' кодами от 'Дочки':")
print(f"  Шеннон-Фано: {len(encoded_put_with_doch_shannon)} бит")
print(f"  Хаффман:     {len(encoded_put_with_doch_huffman)} бит")

print(f"\n'Дочка' кодами от 'Путешествия':")
print(f"  Шеннон-Фано: {len(encoded_doch_with_put_shannon)} бит")
print(f"  Хаффман:     {len(encoded_doch_with_put_huffman)} бит")

print("\nСРАВНЕНИЕ С 'РОДНЫМИ' КОДАМИ:")
print("Текст 'Путешествие':")
print(f"  Родной Шеннон-Фано: {len(encoded_put_shannon)} бит")
print(f"  Чужой Шеннон-Фано:  {len(encoded_put_with_doch_shannon)} бит")
print(f"  Родной Хаффман:     {len(encoded_put_huffman)} бит")
print(f"  Чужой Хаффман:      {len(encoded_put_with_doch_huffman)} бит")

print("\nТекст 'Дочка':")
print(f"  Родной Шеннон-Фано: {len(encoded_doch_shannon)} бит")
print(f"  Чужой Шеннон-Фано:  {len(encoded_doch_with_put_shannon)} бит")
print(f"  Родной Хаффман:     {len(encoded_doch_huffman)} бит")
print(f"  Чужой Хаффман:      {len(encoded_doch_with_put_huffman)} бит")


ПУНКТ 2: РАСПРЕДЕЛЕНИЕ ОТНОСИТЕЛЬНЫХ ЧАСТОТ

 Текст 'Путешествие':
  ' ': 0.1579
  'о': 0.0872
  'е': 0.0724
  'а': 0.0621
  'и': 0.0553
  'н': 0.0525
  'т': 0.0450
  'с': 0.0433
  'л': 0.0367
  'в': 0.0359
  'р': 0.0337
  'д': 0.0278
  'м': 0.0268
  'у': 0.0241
  'к': 0.0232

 Текст 'Дочка':
  ' ': 0.1615
  'о': 0.0794
  'е': 0.0658
  'а': 0.0618
  'и': 0.0518
  'н': 0.0451
  'л': 0.0444
  'т': 0.0435
  'с': 0.0368
  'р': 0.0347
  'м': 0.0288
  'в': 0.0284
  'к': 0.0253
  'у': 0.0234
  'я': 0.0221

ПУНКТ 3: ВЫБОРОЧНАЯ ЭНТРОПИЯ
Энтропия 'Путешествие': 4.5817 бит
Энтропия 'Дочка': 4.7225 бит

ПУНКТ 5: СРЕДНИЕ ДЛИНЫ КОДОВЫХ СЛОВ

 Текст 'Путешествие':
  Равномерный: 7.0000
  Шеннон-Фано: 4.6219
  Хаффман:     4.6180

 Текст 'Дочка':
  Равномерный: 7.0000
  Шеннон-Фано: 4.7721
  Хаффман:     4.7585

ПУНКТ 6: КОДИРОВАНИЕ ТЕКСТОВ

 ТЕКСТ 'ПУТЕШЕСТВИЕ':
Исходная длина: 12713 символов
Равномерный код: 88991 бит (первые 200 бит: 00100000000010001100000011100000001000110000110010000000011100000